In [3]:
import pandas as pd
from smart_open import open

In [10]:
image_net_path = 'http://192.168.4.181:5080/ILSVRC'
train_base_path = f'{image_net_path}/Data/CLS-LOC/train/'
val_base_path = f'{image_net_path}/Data/CLS-LOC/validation/'
train_txt_file = f'{image_net_path}/ImageSets/CLS-LOC/train_cls.txt'
val_index_label_mapping_file = 'http://192.168.4.181:5080/ILSVRC2012_mapping_1000.txt'
val_labels_file = 'http://192.168.4.181:5080/ILSVRC2012_validation_ground_truth.txt'
human_readable_labels_file = 'http://192.168.4.181:5080/imagenet1000_clsidx_to_labels.txt'

In [11]:
with open(train_txt_file) as f:
    train_txt_lines = f.readlines()

In [12]:
with open(val_index_label_mapping_file) as f:
    val_index_label_mapping_lines = (l.split(' ') for l in f.readlines())
    val_index_label_mapping = {index:synset.strip() for index, synset in val_index_label_mapping_lines}

In [13]:
with open(human_readable_labels_file) as f:
    human_readable_labels_str = f.read()
    index_to_human_readable_labels = dict(eval(human_readable_labels_str))

In [14]:
train_base_names = [l.split(' ')[0] for l in train_txt_lines]
train_paths = [f"{train_base_path}{l}.JPEG" for l in train_base_names]
train_synset_labels = [l.split('/')[0] for l in train_base_names]
synset_to_index_label = {label:index for index, label in enumerate(sorted(list(set(train_synset_labels))))}
train_labels = [synset_to_index_label[sl] for sl in train_synset_labels]
train_hr_labels = [index_to_human_readable_labels[l] for l in train_labels]
train_df = pd.DataFrame({
    'path':train_paths, 
    'label':train_labels, 
    'synset_label': train_synset_labels, 
    'human_readable_label':train_hr_labels
})
train_df.to_csv('imagenet_train.csv', index=False)

In [15]:
with open(val_labels_file) as f:
    val_synset_labels = [val_index_label_mapping[l.strip()] for l in f.readlines()]
    val_labels = [synset_to_index_label[sl] for sl in val_synset_labels]
    val_hr_labels = [index_to_human_readable_labels[l] for l in val_labels]
    val_paths = [f'{val_base_path}ILSVRC2012_val_{i+1:08d}.JPEG' for i in range(len(val_labels))]
    val_df = pd.DataFrame({
        'path':val_paths,
        'label':val_labels,
        'synset_label':val_synset_labels,
        'human_readable_label':val_hr_labels
    })
    val_df.to_csv('imagenet_val.csv', index=False)